In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import numpy as np
from pyspark.mllib.linalg.distributed import *
from pyspark.mllib.linalg import *
from scipy.sparse import *
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [2]:
icm = sc.textFile("../icm.csv")
icm = icm.map(lambda l: l.split(','))\
            .filter(lambda line: line[0] != 'itemId')\
            .map(lambda x: (int(x[0]), int(x[1]), 1))
trainSet = sc.textFile("../train.csv")
trainSet = trainSet.map(lambda l: l.split(','))
trainSet = trainSet.filter(lambda line: 'userId' not in line)
trainSet = trainSet.map(lambda line: (int(line[0]), int(line[1]), int(line[2])))
#FOR THE TOP POP
itemSet = trainSet.map(lambda x: (x[1], x[2]))
itemsCount = trainSet.map(lambda x: (x[1],1)).reduceByKey(lambda x,y : x + y)
itemsCount_dict = itemsCount.collectAsMap()
#-----------------------------------------
featureFreq = icm.map(lambda x: (x[1],1)).reduceByKey(lambda x, y: x + y)
featureFreqDict = featureFreq.collectAsMap()
prodCount= icm.map(lambda x: x[0]).distinct().count()
featureIdf = featureFreq.map(lambda x: (x[0],np.log10(prodCount/x[1])))
featureIdfDict = featureIdf.collectAsMap()

targetUsers = sc.textFile("../target_user.csv").filter(lambda x: "userId" not in x).map(lambda x: int(x))
targets=targetUsers.collect()

norms = icm.map(lambda x: (x[0],1))\
                .reduceByKey(lambda x, y: x+y).mapValues(lambda x: np.sqrt(x))\
                .collectAsMap()

normalized = icm.map(lambda x: (x[0], x[1], x[2]/norms[x[0]]))

In [4]:
len(targets)

4196

In [5]:
#Just consider the row of the users to predict
#IF you0re asking, only the idf depends on all the training set, in fact it is computed before reducing the trainset
trainSet=trainSet.filter(lambda x: x[0] in targets)
print(trainSet.count())
data = trainSet.map(lambda x: x[2]).collect()
rows = trainSet.map(lambda x: x[0]).collect()
cols = trainSet.map(lambda x: x[1]).collect()
data.append(0)
rows.append(15364)
cols.append(37142)
userItem=csr_matrix((data,(rows,cols)))
print("userItem shape:",userItem.shape)
data = normalized.map(lambda x: x[2]).collect()
rows = normalized.map(lambda x: x[0]).collect()
cols = normalized.map(lambda x: x[1]).collect()
data.append(0)
rows.append(37142)
cols.append(80)
itemFeature = csc_matrix((data,(rows,cols)))
print("itemFeat shape:",itemFeature.shape)

46750
userItem shape: (15365, 37143)
itemFeat shape: (37143, 19716)


In [6]:
userFeature = userItem.dot(itemFeature)
userFeature.shape

(15365, 19716)

In [7]:
data = []
rows = []
cols = []
for f in featureIdfDict.keys():
    data.append(featureIdfDict[f])
    cols.append(f)
    rows.append(f)
featureIdf = csr_matrix((data,(rows,cols)))
featureIdf.shape

(19716, 19716)

In [8]:
userProfile = userFeature.dot(featureIdf)
prediction = userProfile.dot(itemFeature.transpose())
prediction.shape

(15365, 37143)

In [9]:
print(prediction.getrow(4).toarray()[0][1])
print(prediction.getrow(4).toarray()[0][2])

0.771355602867
2.95764675607


In [10]:
seenItems= trainSet.map(lambda x: (x[0],[x[1]])).reduceByKey(lambda x,y: x + y)
seenItemsDict=seenItems.collectAsMap()
seenItemsDict[100]

[8013, 12255, 30796]

In [11]:
row = prediction.getrow(100).toarray()[0]
user100 = []
for i in range(0,len(row)):
    if i not in seenItemsDict[100]:
        user100.append((i, row[i]))
user100.sort(key = lambda x: -x[1])
print(user100[0])
print(user100[1])
print(user100[2])
print(user100[3])
print(user100[4])

#OK! Works until here

(12535, 18.652665545838573)
(4223, 13.12635525344616)
(11554, 13.12635525344616)
(20532, 13.00817618709717)
(19267, 12.695308889326265)


In [14]:
numberOfRecommendations=5
#TOP POPULAR
cost=7
avgRatings=itemSet.reduceByKey(lambda x,y: x+y)
avgRatings=avgRatings.map(lambda x: (x[0],x[1]/(itemsCount_dict[x[0]]+cost)))
avgRatings.take(5)
itemOrderByPop=avgRatings.sortBy(lambda x: x[1], ascending=False)
itemPop = np.array(itemOrderByPop.map(lambda x: x[0]).collect())
#--------------------------------------------------------------

def recommendTopPop(user_id, removeSeen=True):
    seenItems = np.array(seenItemsDict[user_id])
    recommendedList = itemPop
    if(removeSeen):
        unseen_mask = np.in1d(recommendedList, seenItems, invert=True)
        recommendedList = recommendedList[unseen_mask]       
    return recommendedList[:numberOfRecommendations]


def fillWithTopPop(recommended,user):
    TopPop=recommendTopPop(user)
    for i in range (numberOfRecommendations-len(recommended)):
        recommended.append(TopPop[i])
    return recommended


def getRecommended(user):
    recommended = []
    itemsPred = prediction.getrow(user).toarray()[0]

    for i in range(0,len(itemsPred)):
        if(itemsPred[i]!=0):
                if i not in seenItemsDict[user]:
                    recommended.append((i, row[i]))
    recommended.sort(key = lambda x: -x[1])
    print(recommended[:numberOfRecommendations])
    recommended=recommended[:numberOfRecommendations]
    recommendedItems = list(map(lambda x: x[0], recommended))
    if(len(recommendedItems)<numberOfRecommendations):
        recommendedItems=fillWithTopPop(recommendedItems, user)
    return recommendedItems    



In [15]:
f=open("predictions-opt.csv",'w')
i=0
for user in targets:
    print(user)
    f.write("{}".format(user)+',')
    rec=getRecommended(user)
    print(rec)
    for prod in rec:
        f.write(str(prod) +' ')
    f.write('\n')
    i=i+1
    print(i,"of", len(targets), "written")
f.close()

4
[(30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616), (20532, 13.00817618709717)]
[30796, 12535, 4223, 11554, 20532]
1 of 4196 written
5
[(8013, 27.457348334313153), (30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616)]
[8013, 30796, 12535, 4223, 11554]
2 of 4196 written
8
[(8013, 27.457348334313153), (30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616)]
[8013, 30796, 12535, 4223, 11554]
3 of 4196 written
9
[(8013, 27.457348334313153), (30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616)]
[8013, 30796, 12535, 4223, 11554]
4 of 4196 written
13
[(8013, 27.457348334313153), (21561, 11.4076196781768), (13709, 8.9815093393306711), (2697, 7.7268866990894303), (2176, 7.4139843471360187)]
[8013, 21561, 13709, 2697, 2176]
5 of 4196 written
18
[(8013, 27.457348

146
[(8013, 27.457348334313153), (21561, 11.4076196781768), (13709, 8.9815093393306711), (2697, 7.7268866990894303), (4300, 7.6948596533670415)]
[8013, 21561, 13709, 2697, 4300]
46 of 4196 written
151
[(8013, 27.457348334313153), (30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616)]
[8013, 30796, 12535, 4223, 11554]
47 of 4196 written
152
[(8013, 27.457348334313153), (30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616)]
[8013, 30796, 12535, 4223, 11554]
48 of 4196 written
155
[(30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616), (20532, 13.00817618709717)]
[30796, 12535, 4223, 11554, 20532]
49 of 4196 written
156
[(8013, 27.457348334313153), (21561, 11.4076196781768), (19512, 10.993915450783344), (20870, 10.0463512744535), (13709, 8.9815093393306711)]
[8013, 21561, 19512, 20870, 13709]
50 of 4196 written
160
[(

299
[(8013, 27.457348334313153), (30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616)]
[8013, 30796, 12535, 4223, 11554]
90 of 4196 written
301
[(30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616), (20532, 13.00817618709717)]
[30796, 12535, 4223, 11554, 20532]
91 of 4196 written
308
[(8013, 27.457348334313153), (30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616)]
[8013, 30796, 12535, 4223, 11554]
92 of 4196 written
310
[(30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616), (20532, 13.00817618709717)]
[30796, 12535, 4223, 11554, 20532]
93 of 4196 written
312
[(8013, 27.457348334313153), (21561, 11.4076196781768), (13709, 8.9815093393306711), (2697, 7.7268866990894303), (2176, 7.4139843471360187)]
[8013, 21561, 13709, 2697, 2176]
94 of 4196 written
314
[

443
[(8013, 27.457348334313153), (30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616)]
[8013, 30796, 12535, 4223, 11554]
133 of 4196 written
445
[(8013, 27.457348334313153), (21561, 11.4076196781768), (13709, 8.9815093393306711), (2697, 7.7268866990894303), (4300, 7.6948596533670415)]
[8013, 21561, 13709, 2697, 4300]
134 of 4196 written
449
[(8013, 27.457348334313153), (30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616)]
[8013, 30796, 12535, 4223, 11554]
135 of 4196 written
456
[(8013, 27.457348334313153), (30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616)]
[8013, 30796, 12535, 4223, 11554]
136 of 4196 written
458
[(30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616), (20532, 13.00817618709717)]
[30796, 12535, 4223, 11554, 20532]
137 of 4196 written
4

176 of 4196 written
605
[(8013, 27.457348334313153), (30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616)]
[8013, 30796, 12535, 4223, 11554]
177 of 4196 written
606
[(8013, 27.457348334313153), (30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616)]
[8013, 30796, 12535, 4223, 11554]
178 of 4196 written
615
[(30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616), (20532, 13.00817618709717)]
[30796, 12535, 4223, 11554, 20532]
179 of 4196 written
616
[(8013, 27.457348334313153), (30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616)]
[8013, 30796, 12535, 4223, 11554]
180 of 4196 written
617
[(8013, 27.457348334313153), (30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616)]
[8013, 30796, 12535, 4223, 11554]

...

4002 of 4196 written
14709
[(8013, 27.457348334313153), (30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616)]
[8013, 30796, 12535, 4223, 11554]
4003 of 4196 written
14715
[(30796, 23.836096268859752), (20532, 13.00817618709717), (11133, 11.874785882791105), (21561, 11.4076196781768), (15248, 11.060808754427754)]
[30796, 20532, 11133, 21561, 15248]
4004 of 4196 written
14716
[(30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616), (20532, 13.00817618709717)]
[30796, 12535, 4223, 11554, 20532]
4005 of 4196 written
14718
[(8013, 27.457348334313153), (30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616)]
[8013, 30796, 12535, 4223, 11554]
4006 of 4196 written
14720
[(8013, 27.457348334313153), (30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616)]
[8013, 30796, 1

[(8013, 27.457348334313153), (30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616)]
[8013, 30796, 12535, 4223, 11554]
4046 of 4196 written
14851
[(8013, 27.457348334313153), (30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616)]
[8013, 30796, 12535, 4223, 11554]
4047 of 4196 written
14852
[(21561, 11.4076196781768), (13709, 8.9815093393306711), (2697, 7.7268866990894303), (4300, 7.6948596533670415), (7984, 7.2753417055934602)]
[21561, 13709, 2697, 4300, 7984]
4048 of 4196 written
14854
[(30796, 23.836096268859752), (20532, 13.00817618709717), (11133, 11.874785882791105), (19512, 10.993915450783344), (20870, 10.0463512744535)]
[30796, 20532, 11133, 19512, 20870]
4049 of 4196 written
14856
[(30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616), (20532, 13.00817618709717)]
[30796, 12535, 4223, 11554, 20532]
4050 of 41

4086 of 4196 written
14970
[(8013, 27.457348334313153), (30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616)]
[8013, 30796, 12535, 4223, 11554]
4087 of 4196 written
14973
[(30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616), (20532, 13.00817618709717)]
[30796, 12535, 4223, 11554, 20532]
4088 of 4196 written
14978
[(30796, 23.836096268859752), (20532, 13.00817618709717), (11133, 11.874785882791105), (19512, 10.993915450783344), (20870, 10.0463512744535)]
[30796, 20532, 11133, 19512, 20870]
4089 of 4196 written
14980
[(8013, 27.457348334313153), (21561, 11.4076196781768), (13709, 8.9815093393306711), (2697, 7.7268866990894303), (2176, 7.4139843471360187)]
[8013, 21561, 13709, 2697, 2176]
4090 of 4196 written
14985
[(8013, 27.457348334313153), (30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616)]
[8013, 30796, 125

15111
[(8013, 27.457348334313153), (30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616)]
[8013, 30796, 12535, 4223, 11554]
4129 of 4196 written
15112
[(8013, 27.457348334313153), (30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616)]
[8013, 30796, 12535, 4223, 11554]
4130 of 4196 written
15113
[(30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616), (20532, 13.00817618709717)]
[30796, 12535, 4223, 11554, 20532]
4131 of 4196 written
15114
[(8013, 27.457348334313153), (30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616)]
[8013, 30796, 12535, 4223, 11554]
4132 of 4196 written
15115
[(8013, 27.457348334313153), (21561, 11.4076196781768), (15248, 11.060808754427754), (19512, 10.993915450783344), (20870, 10.0463512744535)]
[8013, 21561, 15248, 19512, 20870]
4133 

4171 of 4196 written
15270
[(8013, 27.457348334313153), (30796, 23.836096268859752), (20532, 13.00817618709717), (11133, 11.874785882791105), (21561, 11.4076196781768)]
[8013, 30796, 20532, 11133, 21561]
4172 of 4196 written
15273
[(30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616), (20532, 13.00817618709717)]
[30796, 12535, 4223, 11554, 20532]
4173 of 4196 written
15275
[(8013, 27.457348334313153), (30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616)]
[8013, 30796, 12535, 4223, 11554]
4174 of 4196 written
15276
[(8013, 27.457348334313153), (21561, 11.4076196781768), (19512, 10.993915450783344), (20870, 10.0463512744535), (13709, 8.9815093393306711)]
[8013, 21561, 19512, 20870, 13709]
4175 of 4196 written
15278
[(8013, 27.457348334313153), (21561, 11.4076196781768), (13709, 8.9815093393306711), (2697, 7.7268866990894303), (4300, 7.6948596533670415)]
[8013, 21561, 1370

In [18]:
print("Result of getRecommended:\n",getRecommended(151))
print("Top 5 from the matrix: " ,prediction.getrow(151).toarray()[0])

[(8013, 27.457348334313153), (30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616)]
Result of getRecommended:
 [8013, 30796, 12535, 4223, 11554]
Top 5 from the matrix:  [ 0.          3.52206684  6.39351585 ...,  0.58138567  4.65589395  0.        ]


In [19]:
print("Result of getRecommended:\n",getRecommended(100))
print("Top 5 from the matrix: " ,prediction.getrow(100).toarray()[0])

[(12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616), (20532, 13.00817618709717), (19267, 12.695308889326265)]
Result of getRecommended:
 [12535, 4223, 11554, 20532, 19267]
Top 5 from the matrix:  [ 0.          2.36576972  3.58037592 ...,  0.          1.72884326  0.        ]
